# Api Get Auctions (EU-Ravencrest)

In [ ]:
# Imports

import pandas as pd
import requests
import re

from datetime import datetime

In [ ]:
# Read access token

with open('../api_access_token.txt', 'r') as file:
   access_token = file.read()

In [ ]:
# Get EU-Ravencrest auctions data

def get_ravencrest(access_token):
   search = f'https://eu.api.blizzard.com/data/wow/connected-realm/1329/auctions?namespace=dynamic-eu&locale=en_GB&access_token={access_token}'
   response = requests.get(search)
   return response.json()['auctions']
 
ravencrest_data = get_ravencrest(access_token)

ravencrest_df = pd.DataFrame(ravencrest_data)

ravencrest_df.head()

In [ ]:
# Add timestamp

now = datetime.now()

date_str = now.strftime('%Y-%m-%d %H:%M:%S')

ravencrest_df['timestamp'] = date_str

ravencrest_df['timestamp'] = pd.to_datetime(ravencrest_df['timestamp'])

ravencrest_df.head()

In [ ]:
# Extract item ids

ravencrest_df['item'] = ravencrest_df['item'].apply(lambda x: x['id'])

ravencrest_df.head()

In [ ]:
# Rename columns

ravencrest_df.rename(columns = {'id': 'auction_id', 'item': 'item_id'}, inplace = True)

ravencrest_df.head()

In [ ]:
# Save ravencrest data to file

def stamp_string(timestamp):
   day = re.findall('-(\d.) ', str(timestamp))[0]
   hour = re.findall(' (\d.):', str(timestamp))[0]
   return 'd' + day + '_h' + hour
   
file_stamp = stamp_string(ravencrest_df.iloc[:1, 6])

ravencrest_df.to_csv(f'../data/auctions/{file_stamp}_auctions.csv', index = False, date_format = '%Y-%m-%d %H:%M:%S')